In [1]:
import basemath as bm
import numpy as np

In [2]:
## Generating input data
## The success metric is binary here:
## Either a visitors turns into a customer (1) or they don't (0)

avg_visitors = 250_000 # average number of visitors per day and variation
cr_A = 0.025 # conversion rate of control
cr_uplift = 0.00 # relative uplift of treatment <- no uplift => cr_A = cr_B
days = 30 # number of simulated days

num_visitors = np.abs(np.random.normal(avg_visitors - 1, avg_visitors * 0.1, days).astype(int)) + 1
customers_A = np.random.binomial(n=num_visitors, p=cr_A)
customers_B = np.random.binomial(n=num_visitors, p=cr_A*(1+cr_uplift))
customers_delta = customers_B - customers_A

In [5]:
## Running Basemath

# input parameters
mean_A = np.sum(customers_A) / np.sum(num_visitors)
mde = 0.01
alpha = 0.05
beta = 0.2
exp_name = "test_experiment_abc"

# initialise
bm_test = bm.BaseMathsTest(mean_A, mde, alpha, beta, seed = exp_name)

# day-by-day evaluation
experiment_state = 0
agg_visitors_seen_before = 0
agg_cust_delta_seen_before = 0
curr_day = 0

for day in range(days):
    experiment_state = bm_test.evaluate_experiment(
        agg_cust_delta_seen_before,
        customers_delta[day],
        agg_visitors_seen_before,
        num_visitors[day]
    )

    curr_day = day
    agg_visitors_seen_before += num_visitors[day]
    agg_cust_delta_seen_before += customers_delta[day]
    
    print("Visitors by day {}:\n {} out of {} ({}%)\n\n"
          .format(
              day + 1,
              agg_visitors_seen_before,
              bm_test.required_samples,
              np.round(agg_visitors_seen_before/bm_test.required_samples*100, 2)
          )
         )

    if experiment_state != 0:
        break

print(experiment_state)
        
# summary
print(f"The experiment evaluation was stopped after {curr_day + 1} days.")
if experiment_state == 1:
    print("Treatment is significantly better than control.")
elif experiment_state == -1:
    print("Treatment is NOT significantly better than control.")
elif experiment_state == 0:
    print("Experiment stopped without conclusion. Not enough data.")
    print(f"{np.sum(num_visitors)} data points per day provided, {bm_test.required_samples} required")

Visitors by day 1:
 269252 out of 5407754 (4.98%)


Visitors by day 2:
 496460 out of 5407754 (9.18%)


Visitors by day 3:
 707792 out of 5407754 (13.09%)


Visitors by day 4:
 1010533 out of 5407754 (18.69%)


Visitors by day 5:
 1257145 out of 5407754 (23.25%)


Visitors by day 6:
 1500542 out of 5407754 (27.75%)


Visitors by day 7:
 1755844 out of 5407754 (32.47%)


-1
The experiment evaluation was stopped after 7 days.
Treatment is NOT significantly better than control.
